# Matrix-vector multiplication via dot products

This notebook walks you through how to implement $ y := A x + y $ via dot products.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with `include` and `using` statements.

## Algorithm

<image src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/341Mvmult_n_unb_var1.png" alt="Matrix-vector multiplication via dot products algorithm" width="75%">

## The `Mvmult_n_unb_var1!( A, x, y )` routine

This routine, given $ A \in \mathbb{R}^{m \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^m $, computes $ y := A x + y $.  The "_n_" indicates this is the "no transpose" matrix-vector multiplication.  What this means will become clear next week.

The specific laff function we will use is
<ul>
<li> <code> laff.dot!( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage </a>.

In [15]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Mvmult_n_unb_var1!(A, x, y)

    AT,
    AB  = flame.part_2x1(A,
                         0, "TOP")

    yT,
    yB  = flame.part_2x1(y,
                         0, "TOP")

    while size(AT, 1) < size(A, 1)

        A0,
        a1t,
        A2   = flame.repart_2x1_to_3x1(AT,
                                       AB,
                                       1, "BOTTOM")

        y0,
        psi1,
        y2    = flame.repart_2x1_to_3x1(yT,
                                        yB,
                                        1, "BOTTOM")

        #------------------------------------------------------------#

        laff.dot!( a1t, x, psi1 )

        #------------------------------------------------------------#

        AT,
        AB  = flame.cont_with_3x1_to_2x1(A0,
                                         a1t,
                                         A2,
                                         "TOP")

        yT,
        yB  = flame.cont_with_3x1_to_2x1(y0,
                                         psi1,
                                         y2,
                                         "TOP")
    end
        
    flame.merge_2x1!(yT,
                    yB, y)
end



Mvmult_n_unb_var1! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 3 matrix and related vectors, performing the computation.

In [16]:
A = rand(4, 3)
x = rand(3)
y = rand(4)
yold = rand(4)

println("A before =")
A

A before =


4×3 Array{Float64,2}:
 0.364776  0.368704    0.437088 
 0.180985  0.0378993   0.0544687
 0.661383  0.00244661  0.325169 
 0.840361  0.028703    0.944274 

In [17]:
println("x before =")
x

x before =


3-element Array{Float64,1}:
 0.9303415412156346
 0.7030511046676859
 0.243355415167549 

In [18]:
println("y before =")
y

y before =


4-element Array{Float64,1}:
 0.56310170304486   
 0.3621085755303912 
 0.17250737815578931
 0.34209393324770065

In [19]:
laff.copy!( y, yold )   # save the original vector y

Mvmult_n_unb_var1!( A, x, y )

println( "y after =" )
y

y after =


4-element Array{Float64,1}:
 1.2680537420022133
 0.5703869035499988
 0.8686709214697254
 1.3738900643857819

In [20]:
println( "y - ( A * x + yold ) = " )
y - ( A * x + yold )

y - ( A * x + yold ) = 


4-element Array{Float64,1}:
 2.220446049250313e-16
 0.0                  
 0.0                  
 0.0                  

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.